In [3]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_read = Path("../Credit_Risk_Classification/Resources/lending_data.csv")
lending_data_df = pd.read_csv(file_read)

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df["loan_status"]

# Separate the X variable, the features
X = lending_data_df.drop(columns="loan_status")

In [6]:
# Review the y variable Series
y.sample(n=5)

18445    0
28666    0
10830    0
73717    0
9373     0
Name: loan_status, dtype: int64

In [7]:
# Review the X variable DataFrame
X.sample(n=5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
39236,9800.0,7.308,49400,0.392713,4,0,19400
15791,9400.0,7.104,47400,0.367089,3,0,17400
58732,9100.0,7.001,46500,0.354839,3,0,16500
74199,9400.0,7.131,47700,0.371069,3,0,17700
71395,8700.0,6.820,44800,0.330357,3,0,14800


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values **Shows the loan_status as imbalanced as more of loans were non-risk as opposed to risk type loans**
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classify_regress = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using training data
classify_regress.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [11]:
# Make a prediction using the testing data
predict = classify_regress.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# Print the balanced_accuracy score of the model 
# ***  The Score of the accuracy is approx. 94%  ***
bal_acc_score = balanced_accuracy_score(y_test, predict)
bal_acc_score

0.9442676901753825

In [13]:
# Generate a confusion matrix for the model
confuse_matrix = confusion_matrix(y_test, predict)
confuse_matrix

array([[18679,    80],
       [   67,   558]])

In [45]:
# put the confusion matrix into an Dataframe for optmize dispplay
confuse_matrix_df = pd.DataFrame(confuse_matrix, 
                                 index=["Actual 0", "Actual 1"], 
                                 columns=["Predicted 0", "Predicted 1"])

In [46]:
# Print the classification report for the model
print("Confusion Matrix: ")
display(confuse_matrix_df)
print("Classification Report: ")
print(classification_report(y_test, predict))

Confusion Matrix: 


,Predicted 0,Predicted 1
Actual 0,18679,80
Actual 1,67,558


Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** `The prediction for the healthy loan (0) shows a great accuracy of 1.0 and for the high risk loan has an accuracy of 0.87`

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [16]:
# installs the imbalanced-learn library (Module Already install)
# %pip install imbalanced-learn

In [31]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ROS = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_sample, y_sample = ROS.fit_resample(X_train, y_train)

In [37]:
# Count the distinct values of the resampled labels data **Now the data is balanced**
print(f'Re sampled dataset shape for {y_sample.value_counts()} ')

Re sampled dataset shape for loan_status
0    56277
1    56277
Name: count, dtype: int64 


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [38]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resample_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
resample_model.fit(X_sample, y_sample)

# Make a prediction using the testing data
resample_model_pred = resample_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [39]:
# Print the balanced_accuracy score of the model
# ***  The Score of the accuracy after using the resampling the training data  is approx. 99%  ***
bal_acc_score_resampled = balanced_accuracy_score(y_test, resample_model_pred)

0.9959744975744975

In [47]:
# Generate a confusion matrix for the model
confuse_matrix_resampled = confusion_matrix(y_test, resample_model_pred)

In [48]:
# set the confusion matrix in Dataframe
confuse_matrix_resampled_df = pd.DataFrame(confuse_matrix_resampled, 
                                           index=["Actual 0", "Actual 1"], 
                                           columns=["Predicted 0", "Predicted 1"])


In [49]:
# Print the classification report for the model
print("Confusion Matrix shows the following:  ")
display(confuse_matrix_resampled_df)
print("Following classification reports shows the following: ")
print(classification_report(y_test, resample_model_pred))

Confusion Matrix shows the following:  


,Predicted 0,Predicted 1
Actual 0,18668,91
Actual 1,2,623


Following classification reports shows the following: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** `After using the RandomOverSampling model showing accuracy of 1.0 for the healthy loan while the high risk loan showing accuracy of 0.87`